In [19]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset

class SensorDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        """
        Args:
            data_dir (str): Path to the directory with the CSV files.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        
        self.data_dir = data_dir
        self.files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_path = os.path.join(self.data_dir, self.files[idx])
        data = pd.read_csv(file_path, delimiter=';')
        data.drop(data.columns[len(data.columns)-1], axis=1, inplace=True)

        X = data.values
        y = torch.Tensor([0, 1]) if '_Fall' in file_path else torch.Tensor([1, 0]) # 0 ADL, 1 Fall
        
        sample = {'features': torch.tensor(X, dtype=torch.float32), 'label': torch.tensor(y, dtype=torch.int32)}

        if self.transform:
            sample = self.transform(sample)

        return sample


In [21]:
from torch import nn

class LSTMClassifier(nn.Module):
    """Simple implementation of LSTM-based time-series classifier."""
    
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.rnn = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.batch_size = None
        self.hidden = None
    
    def forward(self, x):
        h0, c0 = self.init_hidden(x)
        out, (hn, cn) = self.rnn(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out
    
    def init_hidden(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        return [t.cuda() for t in (h0, c0)]

In [22]:
train = SensorDataset(data_dir="../../data/processed/train", transform=None)
valid = SensorDataset(data_dir="../../data/processed/valid", transform=None)

In [23]:
print(train.files)
train.__getitem__(0)

['UMAFall_Subject_16_ADL_GoDownstairs_03_02_04.csv', 'UMAFall_Subject_01_Fall_backwardFall_05_01_01.csv', 'UMAFall_Subject_17_Fall_lateralFall_01_00_01.csv', 'UMAFall_Subject_13_Fall_backwardFall_04_02_02.csv', 'UMAFall_Subject_16_ADL_GoUpstairs_05_02_03.csv', 'UMAFall_Subject_17_Fall_lateralFall_01_00_03.csv', 'UMAFall_Subject_16_ADL_Bending_07_02_01.csv', 'UMAFall_Subject_09_ADL_Bending_03_01_04.csv', 'UMAFall_Subject_16_Fall_lateralFall_05_00_02.csv', 'UMAFall_Subject_16_Fall_lateralFall_11_02_03.csv', 'UMAFall_Subject_03_ADL_Hopping_01_00_01.csv', 'UMAFall_Subject_16_Fall_lateralFall_03_02_02.csv', 'UMAFall_Subject_02_ADL_LyingDown_OnABed_00_04.csv', 'UMAFall_Subject_06_Fall_backwardFall_02_00_03.csv', 'UMAFall_Subject_04_ADL_Hopping_03_02_01.csv', 'UMAFall_Subject_07_ADL_GoDownstairs_02_00_04.csv', 'UMAFall_Subject_04_Fall_lateralFall_04_01_01.csv', 'UMAFall_Subject_14_Fall_forwardFall_02_00_01.csv', 'UMAFall_Subject_09_Fall_backwardFall_03_02_01.csv', 'UMAFall_Subject_16_Fall_lat

/tmp/ipykernel_17426/936033497.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = {'features': torch.tensor(X, dtype=torch.float32), 'label': torch.tensor(y, dtype=torch.int32)}


{'features': tensor([[ 2.8000e+01,  1.0000e+00, -6.9667e+01, -5.5000e+01,  2.7500e+01,
           2.0000e+00],
         [ 2.5700e+02,  2.0000e+00, -6.8500e+01, -5.4667e+01,  2.3667e+01,
           2.0000e+00],
         [ 4.2500e+02,  3.0000e+00, -6.7500e+01, -5.5167e+01,  2.4167e+01,
           2.0000e+00],
         ...,
         [ 1.4989e+04,  2.9400e+02, -1.0083e+02, -2.4808e+01,  2.2315e+01,
           2.0000e+00],
         [ 1.4989e+04,  2.9400e+02, -1.0083e+02, -2.4786e+01,  2.2316e+01,
           2.0000e+00],
         [ 1.4989e+04,  2.9400e+02, -1.0078e+02, -2.4844e+01,  2.2362e+01,
           2.0000e+00]]),
 'label': tensor([1, 0], dtype=torch.int32)}

In [27]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train, batch_size=32, shuffle=True)
valid_loader =  DataLoader(valid, batch_size=32, shuffle=True)

In [32]:
import torch
import torch.nn as nn
import torch.optim
from torch.utils.data import DataLoader

class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMClassifier, self).__init__()
        self.layer_dim = layer_dim
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

input_dim = 6    
hidden_dim = 256
layer_dim = 3
output_dim = 2
seq_dim = 128

lr = 0.0005
n_epochs = 1000
best_acc = 0
patience, trials = 100, 0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LSTMClassifier(input_dim, hidden_dim, layer_dim, output_dim).to(device)
criterion = nn.CrossEntropyLoss()
opt = torch.optim.RMSprop(model.parameters(), lr=lr)

print('Start model training')

for epoch in range(1, n_epochs + 1):
    model.train()
    for i, batch in enumerate(train_loader):
        x_batch = batch["features"].to(device)
        y_batch = batch["label"].to(device)

        opt.zero_grad()
        out = model(x_batch)
        y_batch = y_batch.type(torch.FloatTensor)
        loss = criterion(out, y_batch)
        loss.backward()
        opt.step()
    
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for batch in valid_loader:
            x_val = batch["features"].to(device)
            y_val = batch["label"].to(device)
            out = model(x_val)
            preds = torch.argmax(out, dim=1)
            total += y_val.size(0)
            correct += (preds == y_val).sum().item()
    
    acc = correct / total

    if epoch % 5 == 0:
        print(f'Epoch: {epoch:3d}. Loss: {loss.item():.4f}. Acc.: {acc:2.2%}')

    if acc > best_acc:
        trials = 0
        best_acc = acc
        torch.save(model.state_dict(), 'best.pth')
        print(f'Epoch {epoch} best model saved with accuracy: {best_acc:2.2%}')
    else:
        trials += 1
        if trials >= patience:
            print(f'Early stopping on epoch {epoch}')
            break

Start model training


/tmp/ipykernel_17426/936033497.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = {'features': torch.tensor(X, dtype=torch.float32), 'label': torch.tensor(y, dtype=torch.int32)}
/tmp/ipykernel_17426/936033497.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = {'features': torch.tensor(X, dtype=torch.float32), 'label': torch.tensor(y, dtype=torch.int32)}
/tmp/ipykernel_17426/936033497.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = {'features': torch.tensor(X, dtype=torch.float32), 'label': torch.tensor(

KeyboardInterrupt: 